## Fine tuning model to recognize opened or closed window
Use cases: check if windows are closed if raining or when leaving house


In [22]:
import tensorflow as tf
from tensorflow.keras.applications.mobilenet import MobileNet, preprocess_input
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, GlobalAveragePooling2D
# libraries for displaying images
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import os
import math
from skimage import exposure
from tensorflow.keras.callbacks import CSVLogger

In [23]:
# # Perform all installations
# !pip install tensorflow-gpu==2.5.0
# !pip install tensorflow-datasets
# !pip install tensorwatch
# import tensorflow_datasets as tfds

# tfds makes a lot of progress bars and they take up a lot of screen space, so lets diable them
# tfds.disable_progress_bar()



In [24]:
WIDTH = 224
HEIGHT = 224
NUM_CLASSES = 2
TRAIN_DATA_DIR = r"../contents/windows_imgs/train"
VAL_DATA_DIR = r"../contents/windows_imgs/val"
DATA_DIR = r"../contents/windows_imgs/data"
TEST_DATA_DIR = r"../contents/windows_imgs/test"
LOG_DIR = './log'
TRAIN_SAMPLES = 9
VAL_SAMPLES = 4
BATCH_SIZE = 64
EPOCH = 200
CLASS_THRESHOLD = 0.3
LEARNING_RATE = 0.001

from datetime import datetime
ts = datetime.now()
ts = f"{ts:%Y%m%d-%H%M%S}"


### Prepare the data
load image data from directory and preprocess it

In [25]:

from numpy import left_shift, right_shift


train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    width_shift_range = 0.1,
    height_shift_range = 0.1,
    zoom_range=0.1,
    fill_mode = 'nearest',
    brightness_range=(0.8,1.2),
    # shear_range=0.2,
    # featurewise_center =True,
    # horizontal_flip=True,
    # validation_split=0.1
)

val_datagen = ImageDataGenerator(rescale=1./255)

In [26]:

train_generator = train_datagen.flow_from_directory(
    TRAIN_DATA_DIR,
    target_size=(WIDTH, HEIGHT),
    batch_size = BATCH_SIZE,
    shuffle = True,
    seed = 12345,
    class_mode = 'binary',
    # subset='training',
    save_to_dir=r"../contents/windows_imgs/output/train/",
    save_prefix="",
    save_format='jpg')
TRAIN_SAMPLES = train_generator.samples

val_generator = val_datagen.flow_from_directory(
    VAL_DATA_DIR,
    target_size=(WIDTH, HEIGHT),
    batch_size = BATCH_SIZE,
    shuffle = False,
    class_mode = 'binary',
    # subset='validation',
    save_to_dir=r"../contents/windows_imgs/output/val/",
    save_prefix="",
    save_format='jpg')
VAL_SAMPLES = val_generator.samples


Found 34 images belonging to 2 classes.
Found 32 images belonging to 2 classes.


Code to show the augmented images

In [27]:
# # define the figure plot area
# # fig, ax = plt.subplots(1,NUM_IMG, figsize=(12,4))
# import matplotlib.pyplot as plt

# for i in range(BATCH_SIZE):
#     print(f'Run #{i}')
#     img, label = train_generator.next()
#     # classval = np.argmax(label[0], axis = 0)    # convert one-hot encoding to index
#     # classkey = find_key(classval, iter_img.class_indices) # get class key from index value
#     plt.imshow(img[0])
#     plt.show()
#     # print(f'img shape {img.shape}')
#     # print(f'img label {classkey}\n')

# plt.show()

In [28]:
# import cv2

# # cv2.waitKey(0)
# cam = cv2.VideoCapture(0)

# result, cam_img = cam.read()
# if result:
#     # cv2.imshow("Captured image", cam_img)
#     cv2.imwrite(TEST_DATA_DIR + r"\test_window.jpeg", cam_img)
#     # cv2.waitKey(0)
#     # cv2.destroyAllWindows()
# else:
#     print("No image detected. Please try again.") 


# from sympy import plot


# img_path = r'../contents/windows_imgs/test/test_window.jpeg'
# img = image.load_img(img_path, target_size=(224, 224))
# img_array = image.img_to_array(img)
# expanded_img_array = np.expand_dims(img_array, axis=0)
# preprocessed_img = expanded_img_array / 255.  # Preprocess the image
# prediction = model.predict(preprocessed_img)
# print(prediction)
# print(val_generator.class_indices)

# # plt.figure(1,1)
# plt.imshow(img)
# plt.show()

#### Load the pre-trained modfel and add in fine tuning layers
Now we define the model layers to freeze, discard to classification layers
then finally add in the fine tuning layers


In [29]:

def model_maker():
    # load model
    base_model = MobileNet(
        include_top=False,
        input_shape=(WIDTH, HEIGHT,3))
    for layer in base_model.layers[:]:
        layer.trainable = False
    input = Input(shape=(WIDTH, HEIGHT, 3))
    custom_model = base_model(input)
    custom_model = GlobalAveragePooling2D()(custom_model)
    # custom_model = Flatten()(custom_model)
    custom_model = Dense(256, activation = 'sigmoid')(custom_model)
    custom_model = Dropout(0.5)(custom_model)
    predictions = Dense(1, activation='sigmoid')(custom_model)
    return Model(inputs=input, outputs=predictions)


In [30]:

def model_maker():
    custom_model = Sequential()
    # load model
    base_model = MobileNet(
        include_top=False,
        input_shape=(WIDTH, HEIGHT,3))

    for layer in base_model.layers[:]:
        layer.trainable = False


    for layer in base_model.layers[:-3]:
        layer.trainable = True

    custom_model.add(base_model)
    custom_model.add(GlobalAveragePooling2D())
    # custom_model.add(Flatten())

    # for num in range(num_layers):
    custom_model.add(Dense(640, activation = 'relu'))
    custom_model.add(Dropout(0.3))

    custom_model.add(Dense(1, activation='sigmoid'))


    return custom_model


In [ ]:

def model_maker_MobileNet():
    custom_model = Sequential()
    # load model
    base_model = MobileNet(
        include_top=False,
        input_shape=(WIDTH, HEIGHT,3))

    for layer in base_model.layers[:]:
        layer.trainable = False

    custom_model.add(base_model)
    custom_model.add(GlobalAveragePooling2D())
    custom_model.add(Flatten())
    custom_model.add(Dense(128, activation = 'relu'))
    custom_model.add(Dropout(0.3))

    # use sigmoid for binary classification
    custom_model.add(Dense(1, activation='sigmoid'))

    custom_model.compile(
        loss =  'binary_crossentropy',
        optimizer='adam',
        metrics=['accuracy'])

    return custom_model

Just trying out with another pre-trained model

In [31]:
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2


def model_maker_inceptionResNetV2():
    # load model
    base_model = InceptionResNetV2(
        include_top=False,
        input_shape=(WIDTH, HEIGHT,3))
    for layer in base_model.layers[:]:
        layer.trainable = False
    input = Input(shape=(WIDTH, HEIGHT, 3))
    custom_model = base_model(input)
    custom_model = GlobalAveragePooling2D()(custom_model)
    custom_model = Flatten()(custom_model)
    custom_model = Dense(256, activation = 'relu')(custom_model)
    custom_model = Dropout(0.5)(custom_model)
    predictions = Dense(1, activation='sigmoid')(custom_model)
    return Model(inputs=input, outputs=predictions)


Testing with another model from scratch, however it perform poorly. I am not sure why

In [32]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Convolution2D, MaxPooling2D
# from tensorflow.keras.optimizers import SGD,RMSprop,adam

def model_maker_scratch():
    from_scratch = Sequential()
    from_scratch.add(Convolution2D(224, 3,3,input_shape=(WIDTH, HEIGHT, 3)))
    from_scratch.add(Activation('relu'))
    from_scratch.add(Convolution2D(1000, 3, 3))
    from_scratch.add(Activation('relu'))
    from_scratch.add(MaxPooling2D(pool_size=(2, 2)))
    from_scratch.add(Dropout(0.5))
    # from_scratch.add(Convolution2D(64, 3, 3))
    from_scratch.add(Convolution2D(1000, 3, 3))
    from_scratch.add(Activation('relu'))
    from_scratch.add(MaxPooling2D(pool_size=(2, 2)))
    from_scratch.add(Dropout(0.5))
    # 

    from_scratch.add(Dense(256, activation='relu'))
    from_scratch.add(Dropout(0.5))
    from_scratch.add(Dense(1, activation='sigmoid'))
    # from_scratch.add(Dense(1))
    # from_scratch.add(Activation('sigmoid'))
    return(from_scratch)


In [33]:
# switch between models
# model = model_maker_scratch()
# model = model_maker_inceptionResNetV2()
# model = model_maker()

# model.compile(
#     loss = 'binary_crossentropy',
#     optimizer=tf.keras.optimizers.Adam(LEARNING_RATE),
#     metrics=['accuracy'])

model = model_maker_MobileNet()
model.summary()

from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor = 'val_accuracy', min_delta=0.001, patience=20)

#Allow TensorBoard callbacks
tensorboard_callback = tf.keras.callbacks.TensorBoard(f"{LOG_DIR}\{ts}",
                                                      histogram_freq=1,
                                                      write_graph=True,
                                                      write_grads=True,
                                                      batch_size=BATCH_SIZE,
                                                      write_images=True)

csv_logger = CSVLogger('windows-training-' + 'log.csv',
                        append=True,
                        separator=';')



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenet_1.00_224 (Functi  (None, 7, 7, 1024)        3228864   
 onal)                                                           
                                                                 
 global_average_pooling2d (  (None, 1024)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 640)               656000    
                                                                 
 dropout (Dropout)           (None, 640)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 641       
                                                                 
Total params: 3885505 (14.82 MB)
Trainable params: 28129

In [34]:

from gc import callbacks


history = model.fit(
    train_generator,
    steps_per_epoch=math.ceil(float(TRAIN_SAMPLES)/BATCH_SIZE),
    epochs=EPOCH,
    callbacks=[tensorboard_callback,csv_logger,early_stopping],
    validation_data=val_generator,
    validation_steps=math.ceil(float(VAL_SAMPLES)/BATCH_SIZE))
    

Epoch 1/200
1/1 [==============================] - 13s 13s/step - loss: 0.6979 - accuracy: 0.6176 - val_loss: 2.2020 - val_accuracy: 0.5000
Epoch 2/200
1/1 [==============================] - 6s 6s/step - loss: 1.9519 - accuracy: 0.4412 - val_loss: 0.8176 - val_accuracy: 0.3750
Epoch 3/200
1/1 [==============================] - 6s 6s/step - loss: 0.3226 - accuracy: 0.8824 - val_loss: 1.0602 - val_accuracy: 0.5625
Epoch 4/200
1/1 [==============================] - 6s 6s/step - loss: 0.7427 - accuracy: 0.6765 - val_loss: 0.8378 - val_accuracy: 0.4688
Epoch 5/200
1/1 [==============================] - 6s 6s/step - loss: 0.1652 - accuracy: 0.9118 - val_loss: 1.2038 - val_accuracy: 0.5625
Epoch 6/200
1/1 [==============================] - 6s 6s/step - loss: 0.0995 - accuracy: 0.9706 - val_loss: 1.8649 - val_accuracy: 0.5000
Epoch 7/200
1/1 [==============================] - 6s 6s/step - loss: 0.1245 - accuracy: 0.9412 - val_loss: 2.1229 - val_accuracy: 0.5312
Epoch 8/200
1/1 [===============

In [35]:
# Start TensorBoard
%tensorboard --logdir ./log

UsageError: Line magic function `%tensorboard` not found.


In [ ]:
import matplotlib.pyplot as plt

# Assuming you have a 'history' object with training and validation loss
# Example: history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10)

# Retrieve training and validation loss
train_loss = history.history['loss']
val_loss = history.history['val_loss']

# Create a plot
plt.figure(figsize=(8, 6))
plt.plot(range(1, len(train_loss) + 1), train_loss, label='Training Loss', marker='o')
plt.plot(range(1, len(val_loss) + 1), val_loss, label='Validation Loss', marker='x')

# Customize the plot
plt.title('Training vs. Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.grid(True)
plt.legend()

# Show the plot
plt.show()

In [ ]:
h5_path = f'..\contents\models\winclass_{ts}.h5'
model.save(h5_path)
tf.saved_model.save(model, "tmp/model/1/")

import shutil

# Specify the file you want to copy
source_file = h5_path
destination_file = 'model.h5'

# Copy the file
shutil.copyfile(source_file, destination_file)

print(f"File copied successfully from {source_file} to {destination_file}")

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt

model = load_model('model.h5')

In [ ]:
import cv2

# cv2.waitKey(0)
cam = cv2.VideoCapture(0)

result, cam_img = cam.read()
if result:
    
    # cv2.imshow("Captured image", cam_img)
    # equ_img = cv2.equalizeHist(cam_img)
    cv2.imwrite(TEST_DATA_DIR + r"\test_window.jpeg", cam_img)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
else:
    print("No image detected. Please try again.") 

from skimage import exposure, transform
from skimage.io import imread

img_path = r'../contents/windows_imgs/test/test_window.jpeg'
img = image.load_img(img_path, target_size=(224, 224))

# img = imread(img_path)
# img = exposure.equalize_hist(img)
# new_shape = (224, 224)
# img = transform.resize(img, new_shape, anti_aliasing=True)

img_array = image.img_to_array(img)
expanded_img_array = np.expand_dims(img_array, axis=0)
preprocessed_img = expanded_img_array / 255.  # Preprocess the image
prediction = model.predict(preprocessed_img)
print(prediction)
print(val_generator.class_indices)

# plt.figure(1,1)
plt.title(f"File Name: {img_path}, \n Prediction: Window is  {'Close' if prediction < CLASS_THRESHOLD else 'Open'}")
plt.imshow(img)
plt.show()    


## List all images in test folder and make predictions

In [ ]:
import os
filelist = os.listdir(VAL_DATA_DIR + r"/close/")
# filelist = os.listdir(TEST_DATA_DIR)
jpegfiles = [file for file in filelist if file.endswith('.jpeg')]


Wanted to shrink the images down and display it inside a grid here.. work in progress

In [ ]:
# ## loop through file list and make a grid of images
# cols = 3
# rows = -(-jpegfiles.count // cols)
# fig, axes = plt.subplot(3, rows, figsize(18,6))
# axes = axes.flatten()

# # Plot each image in the corresponding subplot
# for i, image in enumerate(images):
#     img = image.load_img(os.path.join(VAL_DATA_DIR  + r"/close/",file), target_size=(WIDTH,HEIGHT))
#     # img = image.load_img(os.path.join(TEST_DATA_DIR,file), target_size=(WIDTH,HEIGHT))
#     img_array = image.img_to_array(img)
#     expanded_img_array = np.expand_dims(img_array, axis=0)
#     preprocessed_img = expanded_img_array / 255.  # Preprocess the image
#     prediction = model.predict(preprocessed_img)
#     print(prediction)
#     print(val_generator.class_indices)

#     ax = axes[i]
#     ax.imshow(img)
#     ax.title(f"File Name: {file}, \n Prediction: Window is  {'Close' if prediction < CLASS_THRESHOLD else 'Open'}")

#     ax.axis('off')  # Turn off axis labels

# plt.figure()

In [ ]:
for file in jpegfiles:
    img = image.load_img(os.path.join(VAL_DATA_DIR  + r"/close/",file), target_size=(WIDTH,HEIGHT))
    # img = image.load_img(os.path.join(TEST_DATA_DIR,file), target_size=(WIDTH,HEIGHT))
    img_array = image.img_to_array(img)
    expanded_img_array = np.expand_dims(img_array, axis=0)
    preprocessed_img = expanded_img_array / 255.  # Preprocess the image
    prediction = model.predict(preprocessed_img)
    print(prediction)
    print(val_generator.class_indices)

    plt.title(f"File Name: {file}, \n Prediction: Window is  {'Close' if prediction < CLASS_THRESHOLD else 'Open'}")
    plt.imshow(img)
    plt.show()    